In [1]:
import os
import pandas as pd

base_path = "/kaggle/input/datasets/kmader/skin-cancer-mnist-ham10000"

metadata = pd.read_csv("/kaggle/input/datasets/kmader/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")




In [2]:
# Correct base path
base_path = "/kaggle/input/datasets/kmader/skin-cancer-mnist-ham10000/"

image_dir_1 = base_path + "HAM10000_images_part_1/"
image_dir_2 = base_path + "HAM10000_images_part_2/"

def get_image_path(image_id):
    path1 = image_dir_1 + image_id + ".jpg"
    path2 = image_dir_2 + image_id + ".jpg"
    
    if os.path.exists(path1):
        return path1
    else:
        return path2

# Recreate image_path column
metadata["image_path"] = metadata["image_id"].apply(get_image_path)

# Verify first few rows
print(metadata.head())



     lesion_id      image_id   dx dx_type   age   sex localization  \
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp   
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp   
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp   
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp   
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear   

                                          image_path  
0  /kaggle/input/datasets/kmader/skin-cancer-mnis...  
1  /kaggle/input/datasets/kmader/skin-cancer-mnis...  
2  /kaggle/input/datasets/kmader/skin-cancer-mnis...  
3  /kaggle/input/datasets/kmader/skin-cancer-mnis...  
4  /kaggle/input/datasets/kmader/skin-cancer-mnis...  


In [3]:
print(metadata["dx"].value_counts())


dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64


In [4]:
# Create binary label: 1 = melanoma, 0 = others
metadata["label"] = metadata["dx"].apply(lambda x: 1 if x == "mel" else 0)

# Check distribution
print(metadata["label"].value_counts())


label
0    8902
1    1113
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split

# First split: Train (80%) and Test (20%)
train_val_df, test_df = train_test_split(
    metadata,
    test_size=0.2,
    stratify=metadata['label'],
    random_state=42
)

# Second split: Train (70%) and Validation (10%) from remaining
val_relative_size = 0.1 / 0.8  # because 10% of total

train_df, val_df = train_test_split(
    train_val_df,
    test_size=val_relative_size,
    stratify=train_val_df['label'],
    random_state=42
)

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

print("\nTrain distribution:")
print(train_df['label'].value_counts())

print("\nValidation distribution:")
print(val_df['label'].value_counts())

print("\nTest distribution:")
print(test_df['label'].value_counts())


Train size: 7010
Validation size: 1002
Test size: 2003

Train distribution:
label
0    6231
1     779
Name: count, dtype: int64

Validation distribution:
label
0    891
1    111
Name: count, dtype: int64

Test distribution:
label
0    1780
1     223
Name: count, dtype: int64


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64

# Training generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Validation + Test generator (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=True,
    seed=42
)

val_generator = val_test_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_dataframe(
    test_df,
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=False
)


2026-02-16 06:39:20.109920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771223960.289875      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771223960.340469      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771223960.751289      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771223960.751328      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771223960.751331      24 computation_placer.cc:177] computation placer alr

Found 7010 validated image filenames.
Found 1002 validated image filenames.
Found 2003 validated image filenames.


In [7]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(train_df['label'])
weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=train_df['label']
)

class_weights = dict(zip(classes, weights))

print("Class Weights:", class_weights)


Class Weights: {np.int64(0): np.float64(0.5625100304926978), np.int64(1): np.float64(4.499358151476252)}


In [8]:
import tensorflow as tf
print("GPUs Available:", tf.config.list_physical_devices('GPU'))


GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [9]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Enable multi-GPU strategy
strategy = tf.distribute.MirroredStrategy()
print("Number of devices:", strategy.num_replicas_in_sync)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


I0000 00:00:1771223984.794280      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1771223984.800123      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [10]:
with strategy.scope():
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.applications import EfficientNetB0

    base_model = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # IMPORTANT: dtype float32 for stability
    outputs = layers.Dense(1, activation='sigmoid', dtype='float32')(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,383,908 (16.72 MB)

 Trainable params: 331,265 (1.26 MB)

 Non-trainable params: 4,052,643 (15.46 MB)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


callbacks = [
    EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=4,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_auc',
        mode='max',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        '/kaggle/working/binary_best_model.h5',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=12,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

I0000 00:00:1771224007.602030      67 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1771224007.602142      68 cuda_dnn.cc:529] Loaded cuDNN version 91002


110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4812 - auc: 0.4808 - loss: 0.9070
Epoch 1: val_auc improved from -inf to 0.50282, saving model to /kaggle/working/binary_best_model.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.4812 - auc: 0.4808 - loss: 0.9070 - val_accuracy: 0.1108 - val_auc: 0.5028 - val_loss: 0.7452 - learning_rate: 1.0000e-04
Epoch 2/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4971 - auc: 0.5215 - loss: 0.8416
Epoch 2: val_auc improved from 0.50282 to 0.50847, saving model to /kaggle/working/binary_best_model.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.4971 - auc: 0.5214 - loss: 0.8415 - val_accuracy: 0.1108 - val_auc: 0.5085 - val_loss: 0.7114 - learning_rate: 1.0000e-04
Epoch 3/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5203 - auc: 0.5063 - loss: 0.8215
Epoch 3: val_auc improved from 0.50847 to 0.57939, saving model to /kaggle/working/binary_best_model.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 0.5201 - auc: 0.5064 - loss: 0.8216 - val_accuracy: 0.1108 - val_auc: 0.5794 - val_loss: 0.7282 - learning_rate: 1.0000e-04
Epoch 4/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5027 - auc: 0.4983 - loss: 0.8445
Epoch 4: val_auc did not improve from 0.57939
110/110 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.5027 - auc: 0.4984 - loss: 0.8443 - val_accuracy: 0.1108 - val_auc: 0.5000 - val_loss: 0.7337 - learning_rate: 1.0000e-04
Epoch 5/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5148 - auc: 0.5064 - loss: 0.7834
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 5: val_auc did not improve from 0.57939
110/110 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.5148 - auc: 0.5065 - loss: 0.7836 - val_accuracy: 0.1108 - val_auc: 0.5069 - val_loss: 0.7168 - learning_rate: 1.0000e-04
Epoch 6/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5018 - auc: 0.5072 - loss: 0.8097


110/110 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.5018 - auc: 0.5070 - loss: 0.8097 - val_accuracy: 0.1437 - val_auc: 0.6545 - val_loss: 0.7007 - learning_rate: 5.0000e-05
Epoch 7/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5005 - auc: 0.4838 - loss: 0.8151
Epoch 7: val_auc improved from 0.65453 to 0.67720, saving model to /kaggle/working/binary_best_model.h5


110/110 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.5005 - auc: 0.4838 - loss: 0.8152 - val_accuracy: 0.1657 - val_auc: 0.6772 - val_loss: 0.7024 - learning_rate: 5.0000e-05
Epoch 8/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4947 - auc: 0.5439 - loss: 0.7980
Epoch 8: val_auc did not improve from 0.67720
110/110 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.4947 - auc: 0.5438 - loss: 0.7979 - val_accuracy: 0.4900 - val_auc: 0.6761 - val_loss: 0.6936 - learning_rate: 5.0000e-05
Epoch 9/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5125 - auc: 0.5226 - loss: 0.7720
Epoch 9: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

Epoch 9: val_auc did not improve from 0.67720
110/110 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.5124 - auc: 0.5225 - loss: 0.7722 - val_accuracy: 0.1108 - val_auc: 0.6431 - val_loss: 0.7300 - learning_rate: 5.0000e-05
Epoch 10/12
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4975 - auc: 0.5154 - loss: 0.7925

In [12]:
# Unfreeze top 30 layers
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

print("Trainable layers:", len([l for l in model.layers if l.trainable]))


Trainable layers: 9


In [13]:
with strategy.scope():
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )


In [14]:
fine_tune_callbacks = [
    EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_auc',
        mode='max',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=8,
    class_weight=class_weights,
    callbacks=fine_tune_callbacks,
    verbose=1
)


Epoch 1/8
INFO:tensorflow:Collective all_reduce tensors: 36 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 36 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


110/110 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.4926 - auc: 0.5194 - loss: 0.8787 - val_accuracy: 0.8892 - val_auc: 0.6402 - val_loss: 0.4171 - learning_rate: 1.0000e-05
Epoch 2/8
110/110 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.4835 - auc: 0.4957 - loss: 0.8398 - val_accuracy: 0.8892 - val_auc: 0.6721 - val_loss: 0.4021 - learning_rate: 1.0000e-05
Epoch 3/8
110/110 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.5121 - auc: 0.5209 - loss: 0.8375 - val_accuracy: 0.8892 - val_auc: 0.6748 - val_loss: 0.3713 - learning_rate: 1.0000e-05
Epoch 4/8
110/110 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.4905 - auc: 0.5229 - loss: 0.7945 - val_accuracy: 0.8892 - val_auc: 0.6179 - val_loss: 0.3775 - learning_rate: 1.0000e-05
Epoch 5/8
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4961 - auc: 0.5129 - loss: 0.8268
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
110/110 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.4961 - auc: 0.5130 - loss: 0

In [15]:
import pandas as pd

# Convert history to DataFrame
history_df = pd.DataFrame(history.history)

# Save to Kaggle working directory
history_df.to_csv('/kaggle/working/history.csv', index=False)

print("history.csv saved")


history.csv saved


In [16]:
import numpy as np

# Get true labels
y_true = test_generator.labels

# Get predicted probabilities
y_pred_proba = model.predict(test_generator, verbose=1)

# Convert probabilities to class predictions
y_pred = (y_pred_proba.flatten() > 0.5).astype(int)

# Create DataFrame
test_results_df = pd.DataFrame({
    'true_label': y_true,
    'predicted_label': y_pred,
    'probability': y_pred_proba.flatten()
})

# Save file
test_results_df.to_csv('/kaggle/working/test_predictions.csv', index=False)

print("test_predictions.csv saved")


32/32 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step
test_predictions.csv saved


In [17]:
model.save('/kaggle/working/binary_model_final.h5')
print("Model saved successfully")


Model saved successfully
